In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, Activation, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D

In [2]:
noise = np.load("data/data_noise_3.0SNR_1ch_0000.npy").astype(np.float32)
signal = np.load("data/data_signal_B_1ch_0000.npy").astype(np.float32)
n_classes = 2

In [3]:
x = np.vstack((noise, signal))  # shape is (200000, 4, 256)

x2 = np.swapaxes(x, 1, 2)  # (200000, 256, 4)
n_samples = x2.shape[1]
n_channels = x2.shape[2]
x_1D = np.reshape(x, (x.shape[0], -1))  # shape is (200000, 256)
x3 = np.expand_dims(x2, axis=-1)

In [4]:
# define labels with one hot encoding, (1,0) -> noise, (0,1) -> signal
y = np.zeros((200000, 2))
y[:100000, 0] = 1
y[100000:, 1] = 1

y2 = np.zeros(200000)
y2[:100000] = 1

In [5]:
BATCH_SIZE = 32
EPOCHS = 50
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]

In [6]:
model_fc_1layer64  = keras.Sequential(
    [
        Dense(64, activation="relu", name="layer1", input_dim=x_1D.shape[-1]),
        Dense(64, activation="relu", name="layer2"),
        Dense(2, name="layer3"),
    ]
)

In [7]:
model_fc_1layer64.compile(optimizer='Adam',
          loss='binary_crossentropy',
          metrics=['accuracy'])

print(model_fc_1layer64.summary())
model_fc_1layer64.fit(x_1D,y,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          callbacks=callbacks_list,
          validation_split=0.2,
          verbose=1)
model_fc_1layer64.save(f'models/model_fc_1layer64_test.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 64)                16448     
_________________________________________________________________
layer2 (Dense)               (None, 64)                4160      
_________________________________________________________________
layer3 (Dense)               (None, 2)                 130       
Total params: 20,738
Trainable params: 20,738
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
5000/5000 [==============================] - 78s 15ms/step - loss: 0.3359 - accuracy: 0.8897 - val_loss: 0.0985 - val_accuracy: 0.9802
Epoch 2/50
5000/5000 [==============================] - 73s 15ms/step - loss: 0.2023 - accuracy: 0.9692 - val_loss: 0.0873 - val_accuracy: 0.9848
Epoch 3/50
5000/5000 [==============================] - 72s 14ms/step - loss: 0.0313 